# Recommender model building

This notebook is testing out recommendation engines from the Surprise scikit to see if a recommendation system can be made for the Minnesota Institute of Arts data. The Surprise scikit was chosen because of it's adaptable models, thorough documentation, and active repository. More info about Surprise can be found here: https://github.com/NicolasHug/Surprise

Overall, the two models attempting to build a recommendation engine fell far short, and exhibited signs of overfitting. This is likely due to the limited sample size and a tendency of users to submit very few responses. Adding a five-star rating to the responses instead of a simple like/dislike feature is not likely to overcome this data set's limitations. 

In [1]:
import pandas as pd
import os
import random
import numpy as np
from surprise import SVD, KNNBasic, Reader, Dataset, accuracy
from surprise.model_selection import cross_validate, GridSearchCV, train_test_split
from pathlib import Path

my_seed = 23141
random.seed(my_seed)
np.random.seed(my_seed)

In [2]:
dir = Path(os.getcwd())
merged_data = pd.read_csv(dir/"merged_preferences.csv")

In [3]:
merged_data['pref'] = merged_data.pref.replace(False, 0)
merged_data['pref'] = merged_data.pref.replace(True, 1)

Setting the data up to create a collaborative filtering engine, where recommendations are made for a user based on their preference similarities to other users. The surprise scikit requires data to be loaded into its DataSet class to be formatted for model building and prediction. 

In [4]:
user_collab_data = merged_data.iloc[:,0:3]
reader = Reader(rating_scale=(0,1))
ds = Dataset.load_from_df(merged_data[['user_id', 'item_id', 'pref']], reader)

# splitting off a dev set to test after, the train/test split class cannot be used with GridSearchCV
raw_ratings = ds.raw_ratings
random.shuffle(raw_ratings)
threshold = int(.95 * len(raw_ratings))
train_raw_ratings = raw_ratings[:threshold]
dev_raw_ratings = raw_ratings[threshold:]
ds.raw_ratings = train_raw_ratings

First, testing the Singular Value Decomposition system on the data using grid search cross-validation. K-fold cross validation methods split the data into K equal sized datasets, then runs through K iterations of training a model on K-1 combined sets before testing on the remaining set. This reduces bias and variance in the model, helping to prevent a model from overfitting or underfitting to the data. On top of this cross-validation, the Grid Search takes discrete parameter values to be applied to a model and tests each combination of parameters to find the set with the best predictive power. 

The merged preferences have around 3,000 samples, which is a fairly large data set. A 3-fold cross validation will be used on the data, which will leave 20% of the data to be tested on for each fold. The parameters that will be tuned through the Grid Search CV are the number of iterations to reduce the overall error of the model (n_epochs), the learning rate which determines the speed of error minimization (lr_all), and rate of regularization which can prevent model overfitting (lr_all).

In [5]:
grid_search_params = {'n_epochs':[10, 20, 30], 'lr_all': [0.005, 0.010], 'reg_all': [0.01, .02, .03]}
svd_grid = GridSearchCV(SVD, grid_search_params, measures=['rmse', 'mae'], cv=3)
svd_grid.fit(ds)
print(svd_grid.best_score['rmse'])
print(svd_grid.best_params['rmse'])

svd_model = svd_grid.best_estimator['rmse']
train_set = ds.build_full_trainset()
svd_model.fit(train_set)
predictions = svd_model.test(train_set.build_testset())
svd_dev = accuracy.rmse(predictions)
print(f'Biased accuracy on dev set: {svd_dev}')

0.451384493228673
{'n_epochs': 10, 'lr_all': 0.01, 'reg_all': 0.02}
RMSE: 0.3327
Biased accuracy on dev set: 0.33272284582328043


The SCD system provided some fairly low estimations overall, with the best values at 45% and a very poor performance on the validation set at 33% correct recommendation. This is very likely due to overfitting the model, which can be corrected by collecting more data. We could try to continue refining this model by refining grid search parameters basic on the best score model, but let's see if the K-nearest neighbor algorithm has better results first. 

In [6]:
grid_search_KNN_params = {'k': [10, 20, 30, 40, 50, 60], 'min_k': [2, 3, 4, 5]}
knn_grid = GridSearchCV(KNNBasic, grid_search_KNN_params, measures=['rmse', 'mae'], cv=5)
knn_grid.fit(ds)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


In [7]:
print(knn_grid.best_score['rmse'])
print(knn_grid.best_params['rmse'])

knn_model = knn_grid.best_estimator['rmse']
train_set = ds.build_full_trainset()
knn_model.fit(train_set)
predictions = knn_model.test(train_set.build_testset())
knn_dev = accuracy.rmse(predictions)
print(f'Biased accuracy on dev set: {knn_dev}')

0.4623614024073455
{'k': 20, 'min_k': 5}
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.2741
Biased accuracy on dev set: 0.274098716095245


Similar results as the SVD model, seems to be highly overfit. This can be overcome by gathering a lot more data, and isn't wholly unexpected since there are 181 unique pieces of art submitted by only 757 users, with only around 80% of users submitting more than one response (see the data analysis notebook for more details). Trying out a KNN model that focuses on the artwork instead of the users, but the results are very similar.

In [8]:
algo = KNNBasic(k=30, min_k=5, sim_options= {'name': 'msd', 'user_based': 'False'})
cross_validate(algo, ds, measures=['rmse'], cv=5)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([0.45525912, 0.46087442, 0.47006994, 0.48130939, 0.45791226]),
 'fit_time': (0.04118037223815918,
  0.03381776809692383,
  0.016953468322753906,
  0.017078876495361328,
  0.01998734474182129),
 'test_time': (0.04884767532348633,
  0.04395866394042969,
  0.048543453216552734,
  0.04515337944030762,
  0.04697418212890625)}